# Hello Tergite

This is a showcase of connecting to tergite via the `tergite-qiskit-connector` and running a basic circuit.

## Import the basic dependencies

In [1]:
import qiskit.circuit as circuit
import qiskit.compiler as compiler
from tergite_qiskit_connector.providers.tergite import Tergite
from tergite_qiskit_connector.providers.tergite.provider_account import ProviderAccount

## Update Some Variables

In [6]:
# the Tergite API URL e.g. "https://api.tergite.example"
API_URL = "https://api.qal9000.se"
# API token for connecting to tergite. Required if no username/password
API_TOKEN = "<your Tergite API key >"
# API username, required if API_TOKEN is not set
API_USERNAME = "<your API username>"
# API password, required if API_USERNAME is set
API_PASSWORD = "<your API password>"
# The name of the Quantum Computer to use from the available quantum computers
BACKEND_NAME = "Loke"
# the name of this service. For your own bookkeeping.
SERVICE_NAME = "local"

## Create the Qiskit Circuit

In [ ]:
qc = circuit.QuantumCircuit(2, 2)
qc.h(1)
qc.measure(1, 1)
qc.draw()

## Get the Tergite Backend

In [ ]:
 # create a provider
if API_USERNAME:
    account_extras = {"username": API_USERNAME, "password": API_PASSWORD}
else:
    account_extras = {}

# provider account creation can be skipped in case you already saved
# your provider account to the `~/.qiskit/tergiterc` file.
# See below how that is done.
account = ProviderAccount(
        service_name=SERVICE_NAME, url=API_URL, token=API_TOKEN, extras=account_extras
)

provider = Tergite.use_provider_account(account)
# to save this account to the `~/.qiskit/tergiterc` file, add the `save=True`
# provider = Tergite.use_provider_account(account, save=True)

# Get the tergite backend in case you skipped provider account creation
# provider = Tergite.get_provider(service_name=SERVICE_NAME)
backend = provider.get_backend(BACKEND_NAME)
backend.set_options(shots=1024)
backend

## Compile Circuit

In [ ]:
tc = compiler.transpile(qc, backend = backend)
tc.draw()

## Run the Circuit

In [6]:
job = backend.run(tc, meas_level = 2)

Tergite: Job has been successfully submitted


## See the Results

In [ ]:
job.result()